In [3]:
# Install the Client library (Semantic Retriever is only supported for versions >0.4.0)
!pip3 install -U google.ai.generativelanguage

     -------------------------------------- 719.2/719.2 kB 3.0 MB/s eta 0:00:00
     -------------------------------------- 139.4/139.4 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 195.5/195.5 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 50.1/50.1 kB ? eta 0:00:00
     -------------------------------------- 426.9/426.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.3
    Uninstalling protobuf-4.25.3:
      Successfully uninstalled protobuf-4.25.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.25.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.27.2 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install -U google-auth-oauthlib

     -------------------------------------- 151.7/151.7 kB 3.0 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
service_account_file_name = '../contradiction-checker-583a085c67ca.json'

from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(service_account_file_name)

scoped_credentials = credentials.with_scopes(
    ['https://www.googleapis.com/auth/cloud-platform', 'https://www.googleapis.com/auth/generative-language.retriever'])

In [7]:
import google.ai.generativelanguage as glm
generative_service_client = glm.GenerativeServiceClient(credentials=scoped_credentials)
retriever_service_client = glm.RetrieverServiceClient(credentials=scoped_credentials)
permission_service_client = glm.PermissionServiceClient(credentials=scoped_credentials)

In [8]:
example_corpus = glm.Corpus(display_name="Google for Developers Blog")
create_corpus_request = glm.CreateCorpusRequest(corpus=example_corpus)

# Make the request
create_corpus_response = retriever_service_client.create_corpus(create_corpus_request)

# Set the `corpus_resource_name` for subsequent sections.
corpus_resource_name = create_corpus_response.name
print(create_corpus_response)

name: "corpora/google-for-developers-blog-iiiol6kl1i8x"
display_name: "Google for Developers Blog"
create_time {
  seconds: 1720534394
  nanos: 427855000
}
update_time {
  seconds: 1720534394
  nanos: 427855000
}



In [9]:
get_corpus_request = glm.GetCorpusRequest(name=corpus_resource_name)

# Make the request
get_corpus_response = retriever_service_client.get_corpus(get_corpus_request)

# Print the response
print(get_corpus_response)

name: "corpora/google-for-developers-blog-iiiol6kl1i8x"
display_name: "Google for Developers Blog"
create_time {
  seconds: 1720534394
  nanos: 427855000
}
update_time {
  seconds: 1720534394
  nanos: 427855000
}



In [10]:
# Create a document with a custom display name.
example_document = glm.Document(display_name="Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development")

# Add metadata.
# Metadata also supports numeric values not specified here
document_metadata = [
    glm.CustomMetadata(key="url", string_value="https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html")]
example_document.custom_metadata.extend(document_metadata)

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
create_document_request = glm.CreateDocumentRequest(parent=corpus_resource_name, document=example_document)
create_document_response = retriever_service_client.create_document(create_document_request)

# Set the `document_resource_name` for subsequent sections.
document_resource_name = create_document_response.name
print(create_document_response)

name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3"
display_name: "Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development"
custom_metadata {
  key: "url"
  string_value: "https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html"
}
update_time {
  seconds: 1720534528
  nanos: 859633000
}
create_time {
  seconds: 1720534528
  nanos: 859633000
}



In [11]:
get_document_request = glm.GetDocumentRequest(name=document_resource_name)

# Make the request
# document_resource_name is a variable set in the "Create a document" section.
get_document_response = retriever_service_client.get_document(get_document_request)

# Print the response
print(get_document_response)

name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3"
display_name: "Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development"
custom_metadata {
  key: "url"
  string_value: "https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html"
}
update_time {
  seconds: 1720534528
  nanos: 859633000
}
create_time {
  seconds: 1720534528
  nanos: 859633000
}



In [12]:
!pip install google-labs-html-chunker

from google_labs_html_chunker.html_chunker import HtmlChunker

from urllib.request import urlopen

  Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
     -------------------------------------- 112.2/112.2 kB 2.2 MB/s eta 0:00:00
  Using cached soupsieve-2.5-py3-none-any.whl (36 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
with(urlopen("https://developers.googleblog.com/2023/08/introducing-project-idx-experiment-to-improve-full-stack-multiplatform-app-development.html")) as f:
  html = f.read().decode("utf-8")

In [14]:
# Chunk the file using HtmlChunker
chunker = HtmlChunker(
    max_words_per_aggregate_passage=200,
    greedily_aggregate_sibling_nodes=True,
    html_tags_to_exclude={"noscript", "script", "style"},
)
passages = chunker.chunk(html)
print(passages)


# Create `Chunk` entities.
chunks = []
for passage in passages:
    chunk = glm.Chunk(data={'string_value': passage})
    # Optionally, you can add metadata to a chunk
    chunk.custom_metadata.append(glm.CustomMetadata(key="tags",
                                                    string_list_value=glm.StringList(
                                                        values=["Google For Developers", "Project IDX", "Blog", "Announcement"])))
    chunk.custom_metadata.append(glm.CustomMetadata(key="chunking_strategy",
                                                    string_value="greedily_aggregate_sibling_nodes"))
    chunk.custom_metadata.append(glm.CustomMetadata(key = "publish_date",
                                                    numeric_value = 20230808))
    chunks.append(chunk)
print(chunks)

['Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development\n            \n            \n            \n            - Google Developers Blog', 'Products Develop Android Chrome ChromeOS Cloud Firebase Flutter Google Assistant Google Maps Platform Google Workspace TensorFlow YouTube Grow Firebase Google Ads Google Analytics Google Play Search Web Push and Notification APIs Earn AdMob Google Ads API Google Pay Google Play Billing Interactive Media Ads Solutions Events Learn Community Groups Google Developer Groups Google Developer Student Clubs Woman Techmakers Google Developer Experts Tech Equity Collective Programs Accelerator Solution Challenge DevFest Stories All Stories Developer Program Blog Search English English Español (Latam) Bahasa Indonesia 日本語 한국어 Português (Brasil) 简体中文', 'Products More Solutions Events Learn Community More Developer Program Blog Develop Android Chrome ChromeOS Cloud Firebase Flutter Google Assistant Google Maps Platform Goog

In [15]:
# Option 1: Use HtmlChunker in the section above.
# `chunks` is the variable set from the section above.
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))

# Make the request
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

chunks {
  name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/zy2f8m98kbux"
  data {
    string_value: "Introducing Project IDX, An Experiment to Improve Full-stack, Multiplatform App Development\n            \n            \n            \n            - Google Developers Blog"
  }
  custom_metadata {
    key: "tags"
    string_list_value {
      values: "Google For Developers"
      values: "Project IDX"
      values: "Blog"
      values: "Announcement"
    }
  }
  custom_metadata {
    key: "chunking_strategy"
    string_value: "greedily_aggregate_sibling_nodes"
  }
  custom_metadata {
    key: "publish_date"
    numeric_value: 20230808
  }
  state: STATE_PENDING_PROCESSING
}
chunks {
  name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/jy8pcszi20h3"
  data {
    string_value: "Products Develop Android Chrome ChromeOS Cloud Firebase Flutter Google Assistant Google

In [16]:
# Add up to 100 CreateChunk requests per batch request.
# document_resource_name is a variable set in the "Create a document" section.
chunks = []
chunk_1 = glm.Chunk(data={'string_value': "Chunks support user specified metadata."})
chunk_1.custom_metadata.append(glm.CustomMetadata(key="section",
                                                  string_value="Custom metadata filters"))
chunk_2 = glm.Chunk(data={'string_value': "The maximum number of metadata supported is 20"})
chunk_2.custom_metadata.append(glm.CustomMetadata(key = "num_keys",
                                                  numeric_value = 20))
chunks = [chunk_1, chunk_2]
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))

# Make the request
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

chunks {
  name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/v43tyvokpegt"
  data {
    string_value: "Chunks support user specified metadata."
  }
  custom_metadata {
    key: "section"
    string_value: "Custom metadata filters"
  }
  state: STATE_PENDING_PROCESSING
}
chunks {
  name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/8w1vzhfaa8c8"
  data {
    string_value: "The maximum number of metadata supported is 20"
  }
  custom_metadata {
    key: "num_keys"
    numeric_value: 20
  }
  state: STATE_PENDING_PROCESSING
}



In [17]:
# Make the request
request = glm.ListChunksRequest(parent=document_resource_name)
list_chunks_response = retriever_service_client.list_chunks(request)
for index, chunks in enumerate(list_chunks_response.chunks):
  print(f'\nChunk # {index + 1}')
  print(f'Resource Name: {chunks.name}')
  # Only ACTIVE chunks can be queried.
  print(f'State: {glm.Chunk.State(chunks.state).name}')


Chunk # 1
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/4z8b1269fayt
State: STATE_ACTIVE

Chunk # 2
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/6tvx8kgacs0d
State: STATE_ACTIVE

Chunk # 3
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/8c9p6upvq0m1
State: STATE_ACTIVE

Chunk # 4
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/bu4fum8h35tc
State: STATE_ACTIVE

Chunk # 5
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/dmmm49ujud3o
State: STATE_ACTIVE

Chunk # 6
Resource Name: corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/gg9c2o5cclmx
State: STATE_ACTIVE

Chunk # 7

In [18]:
# Create a document with a custom display name.
example_document = glm.Document(display_name="How it’s Made: Interacting with Gemini through multimodal prompting")

# Add document metadata.
# Metadata also supports numeric values not specified here
document_metadata = [
    glm.CustomMetadata(key="url", string_value="https://developers.googleblog.com/2023/12/how-its-made-gemini-multimodal-prompting.html")]
example_document.custom_metadata.extend(document_metadata)

# Make the CreateDocument request
# corpus_resource_name is a variable set in the "Create a corpus" section.
create_document_request = glm.CreateDocumentRequest(parent=corpus_resource_name, document=example_document)
create_document_response = retriever_service_client.create_document(create_document_request)

# Set the `document_resource_name` for subsequent sections.
document_resource_name = create_document_response.name
print(create_document_response)

# Chunks - add another webpage from Google for Developers
with(urlopen("https://developers.googleblog.com/2023/12/how-its-made-gemini-multimodal-prompting.html")) as f:
  html = f.read().decode("utf-8")

# Chunk the file using HtmlChunker
chunker = HtmlChunker(
    max_words_per_aggregate_passage=100,
    greedily_aggregate_sibling_nodes=False,
)
passages = chunker.chunk(html)

# Create `Chunk` entities.
chunks = []
for passage in passages:
    chunk = glm.Chunk(data={'string_value': passage})
    chunk.custom_metadata.append(glm.CustomMetadata(key="tags",
                                                    string_list_value=glm.StringList(
                                                        values=["Google For Developers", "Gemini API", "Blog", "Announcement"])))
    chunk.custom_metadata.append(glm.CustomMetadata(key="chunking_strategy",
                                                    string_value="no_aggregate_sibling_nodes"))
    chunk.custom_metadata.append(glm.CustomMetadata(key = "publish_date",
                                                    numeric_value = 20231206))
    chunks.append(chunk)

# Make the request
create_chunk_requests = []
for chunk in chunks:
  create_chunk_requests.append(glm.CreateChunkRequest(parent=document_resource_name, chunk=chunk))
request = glm.BatchCreateChunksRequest(parent=document_resource_name, requests=create_chunk_requests)
response = retriever_service_client.batch_create_chunks(request)
print(response)

name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/how-its-made-interacting-wi-xtfkq8f4eabt"
display_name: "How it’s Made: Interacting with Gemini through multimodal prompting"
custom_metadata {
  key: "url"
  string_value: "https://developers.googleblog.com/2023/12/how-its-made-gemini-multimodal-prompting.html"
}
update_time {
  seconds: 1720534640
  nanos: 397824000
}
create_time {
  seconds: 1720534640
  nanos: 397824000
}

chunks {
  name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/how-its-made-interacting-wi-xtfkq8f4eabt/chunks/es3xonq76mhc"
  data {
    string_value: "How it’s Made: Interacting with Gemini through multimodal prompting\n            \n            \n            \n            - Google Developers Blog"
  }
  custom_metadata {
    key: "tags"
    string_list_value {
      values: "Google For Developers"
      values: "Gemini API"
      values: "Blog"
      values: "Announcement"
    }
  }
  custom_metadata {
    key: "chunking_strategy"
  

In [21]:
user_query = "What is the purpose of Project IDX?"
results_count = 5

# Add metadata filters for both chunk and document.
chunk_metadata_filter = glm.MetadataFilter(key='chunk.custom_metadata.tags',
                                           conditions=[glm.Condition(
                                              string_value='Google For Developers',
                                              operation=glm.Condition.Operator.INCLUDES)])

# Make the request
# corpus_resource_name is a variable set in the "Create a corpus" section.
request = glm.QueryCorpusRequest(name=corpus_resource_name,
                                 query=user_query,
                                 results_count=results_count,
                                 metadata_filters=[chunk_metadata_filter])
query_corpus_response = retriever_service_client.query_corpus(request)
print(query_corpus_response)

relevant_chunks {
  chunk_relevance_score: 0.750315785
  chunk {
    name: "corpora/google-for-developers-blog-iiiol6kl1i8x/documents/introducing-project-idx-an--1dk0qr1hsqx3/chunks/hpbpzavvgdof"
    data {
      string_value: "Project IDX is a browser-based development experience built on Google Cloud and powered by Codey , a foundational AI model trained on code and built on PaLM 2. It’s designed to make it easier to build, manage and deploy full-stack web and multiplatform applications, with popular frameworks and languages. Project IDX is also built on Code OSS , so it should feel familiar no matter what you’re building. A big part of why we’re sharing Project IDX today is we’d love to hear from the broader developer community on what could help you work even faster. In the meantime, here’s a preview of what’s possible today with Project IDX. Get to work quickly, from anywhere At the heart of Project IDX is our conviction that you should be able to develop from anywhere, on any dev